## Week 4, Lab 2: Predicting Chronic Kidney Disease in Patients
> Author: Matt Brems

We can sketch out the data science process as follows:
1. Define the problem.
2. Obtain the data.
3. Explore the data.
4. Model the data.
5. Evaluate the model.
6. Answer the problem.

In this lab, we're going to focus on steps exploring data, building models and evaluating the models we build.

There are three links you may find important:
- [A set of chronic kidney disease (CKD) data and other biological factors](./chronic_kidney_disease_full.csv).
- [The CKD data dictionary](./chronic_kidney_disease_header.txt).
- [An article comparing the use of k-nearest neighbors and support vector machines on predicting CKD](./chronic_kidney_disease.pdf).

## Step 1: Define the problem.

Suppose you're working for Mayo Clinic, widely recognized to be the top hospital in the United States. In your work, you've overheard nurses and doctors discuss test results, then arrive at a conclusion as to whether or not someone has developed a particular disease or condition. For example, you might overhear something like:

> **Nurse**: Male 57 year-old patient presents with severe chest pain. FDP _(short for fibrin degradation product)_ was elevated at 13. We did an echo _(echocardiogram)_ and it was inconclusive.

> **Doctor**: What was his interarm BP? _(blood pressure)_

> **Nurse**: Systolic was 140 on the right; 110 on the left.

> **Doctor**: Dammit, it's an aortic dissection! Get to the OR _(operating room)_ now!

> _(intense music playing)_

In this fictitious but [Shonda Rhimes-esque](https://en.wikipedia.org/wiki/Shonda_Rhimes#Grey's_Anatomy,_Private_Practice,_Scandal_and_other_projects_with_ABC) scenario, you might imagine the doctor going through a series of steps like a [flowchart](https://en.wikipedia.org/wiki/Flowchart), or a series of if-this-then-that steps to diagnose a patient. The first steps made the doctor ask what the interarm blood pressure was. Because interarm blood pressure took on the values it took on, the doctor diagnosed the patient with an aortic dissection.

Your goal, as a research biostatistical data scientist at the nation's top hospital, is to develop a medical test that can improve upon our current diagnosis system for [chronic kidney disease (CKD)](https://www.mayoclinic.org/diseases-conditions/chronic-kidney-disease/symptoms-causes/syc-20354521).

**Real-world problem**: Develop a medical diagnosis test that is better than our current diagnosis system for CKD.

**Data science problem**: Develop a medical diagnosis test that reduces both the number of false positives and the number of false negatives.

---

## Step 2: Obtain the data.

### 1. Read in the data.

In [32]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

df = pd.read_csv('chronic_kidney_disease_full.csv')
df.head()

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wbcc,rbcc,htn,dm,cad,appet,pe,ane,class
0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,121.0,...,44.0,7800.0,5.2,yes,yes,no,good,no,no,ckd
1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,NaN,...,38.0,6000.0,NaN,no,no,no,good,no,no,ckd
2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,423.0,...,31.0,7500.0,NaN,no,yes,no,poor,no,yes,ckd
3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,...,32.0,6700.0,3.9,yes,no,no,poor,yes,yes,ckd
4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,106.0,...,35.0,7300.0,4.6,no,no,no,good,no,no,ckd


### 2. Check out the data dictionary. What are a few features or relationships you might be interested in checking out?

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 25 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   age     391 non-null    float64
 1   bp      388 non-null    float64
 2   sg      353 non-null    float64
 3   al      354 non-null    float64
 4   su      351 non-null    float64
 5   rbc     248 non-null    object 
 6   pc      335 non-null    object 
 7   pcc     396 non-null    object 
 8   ba      396 non-null    object 
 9   bgr     356 non-null    float64
 10  bu      381 non-null    float64
 11  sc      383 non-null    float64
 12  sod     313 non-null    float64
 13  pot     312 non-null    float64
 14  hemo    348 non-null    float64
 15  pcv     329 non-null    float64
 16  wbcc    294 non-null    float64
 17  rbcc    269 non-null    float64
 18  htn     398 non-null    object 
 19  dm      398 non-null    object 
 20  cad     398 non-null    object 
 21  appet   399 non-null    object 
 22  pe

Answer: age and bp are a few features I am interested in.

---

## Step 3: Explore the data.

### 3. How much of the data is missing from each column?

In [34]:
df.isnull().sum().sort_values(ascending=False)

rbc      152
rbcc     131
wbcc     106
pot       88
sod       87
pcv       71
pc        65
hemo      52
su        49
sg        47
al        46
bgr       44
bu        19
sc        17
bp        12
age        9
ba         4
pcc        4
htn        2
dm         2
cad        2
appet      1
pe         1
ane        1
class      0
dtype: int64

### 4. Suppose that I dropped every row that contained at least one missing value. (In the context of analysis with missing data, we call this a "complete case analysis," because we keep only the complete cases!) How many rows would remain in our dataframe? What are at least two downsides to doing this?

> There's a good visual on slide 15 of [this deck](https://liberalarts.utexas.edu/prc/_files/cs/Missing-Data.pdf) that shows what a complete case analysis looks like if you're interested.

In [35]:
df_dropna = df.dropna(axis=0)
df_dropna

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wbcc,rbcc,htn,dm,cad,appet,pe,ane,class
3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,...,32.0,6700.0,3.9,yes,no,no,poor,yes,yes,ckd
9,53.0,90.0,1.020,2.0,0.0,abnormal,abnormal,present,notpresent,70.0,...,29.0,12100.0,3.7,yes,yes,no,poor,no,yes,ckd
11,63.0,70.0,1.010,3.0,0.0,abnormal,abnormal,present,notpresent,380.0,...,32.0,4500.0,3.8,yes,yes,no,poor,yes,no,ckd
14,68.0,80.0,1.010,3.0,2.0,normal,abnormal,present,present,157.0,...,16.0,11000.0,2.6,yes,yes,yes,poor,yes,no,ckd
20,61.0,80.0,1.015,2.0,0.0,abnormal,abnormal,notpresent,notpresent,173.0,...,24.0,9200.0,3.2,yes,yes,yes,poor,yes,yes,ckd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,55.0,80.0,1.020,0.0,0.0,normal,normal,notpresent,notpresent,140.0,...,47.0,6700.0,4.9,no,no,no,good,no,no,notckd
396,42.0,70.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,75.0,...,54.0,7800.0,6.2,no,no,no,good,no,no,notckd
397,12.0,80.0,1.020,0.0,0.0,normal,normal,notpresent,notpresent,100.0,...,49.0,6600.0,5.4,no,no,no,good,no,no,notckd
398,17.0,60.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,114.0,...,51.0,7200.0,5.9,no,no,no,good,no,no,notckd


Answer: 158 rows will remain. 
        1) There are 400 rows in this dataframe, 242 rows dropped would constitute 60% of our data, a sizeable amount.
        2) We will be dropping rows with missing data in features that will not be used in our analysis

### 5. Thinking critically about how our data were gathered, it's likely that these records were gathered by doctors and nurses. Brainstorm three potential areas (in addition to the missing data we've already discussed) where this data might be inaccurate or imprecise.

In [36]:
df.select_dtypes(include='object')

,rbc,pc,pcc,ba,htn,dm,cad,appet,pe,ane,class
0,NaN,normal,notpresent,notpresent,yes,yes,no,good,no,no,ckd
1,NaN,normal,notpresent,notpresent,no,no,no,good,no,no,ckd
2,normal,normal,notpresent,notpresent,no,yes,no,poor,no,yes,ckd
3,normal,abnormal,present,notpresent,yes,no,no,poor,yes,yes,ckd
4,normal,normal,notpresent,notpresent,no,no,no,good,no,no,ckd
...,...,...,...,...,...,...,...,...,...,...,...
395,normal,normal,notpresent,notpresent,no,no,no,good,no,no,notckd
396,normal,normal,notpresent,notpresent,no,no,no,good,no,no,notckd
397,normal,normal,notpresent,notpresent,no,no,no,good,no,no,notckd
398,normal,normal,notpresent,notpresent,no,no,no,good,no,no,notckd


In [37]:
df.select_dtypes(exclude='object')

,age,bp,sg,al,su,bgr,bu,sc,sod,pot,hemo,pcv,wbcc,rbcc
0,48.0,80.0,1.020,1.0,0.0,121.0,36.0,1.2,NaN,NaN,15.4,44.0,7800.0,5.2
1,7.0,50.0,1.020,4.0,0.0,NaN,18.0,0.8,NaN,NaN,11.3,38.0,6000.0,NaN
2,62.0,80.0,1.010,2.0,3.0,423.0,53.0,1.8,NaN,NaN,9.6,31.0,7500.0,NaN
3,48.0,70.0,1.005,4.0,0.0,117.0,56.0,3.8,111.0,2.5,11.2,32.0,6700.0,3.9
4,51.0,80.0,1.010,2.0,0.0,106.0,26.0,1.4,NaN,NaN,11.6,35.0,7300.0,4.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,55.0,80.0,1.020,0.0,0.0,140.0,49.0,0.5,150.0,4.9,15.7,47.0,6700.0,4.9
396,42.0,70.0,1.025,0.0,0.0,75.0,31.0,1.2,141.0,3.5,16.5,54.0,7800.0,6.2
397,12.0,80.0,1.020,0.0,0.0,100.0,26.0,0.6,137.0,4.4,15.8,49.0,6600.0,5.4
398,17.0,60.0,1.025,0.0,0.0,114.0,50.0,1.0,135.0,4.9,14.2,51.0,7200.0,5.9


Answer: 1) categorical columns can be subjective
        2) 

---

## Step 4: Model the data.

### 6. Suppose that I want to construct a model where no person who has CKD will ever be told that they do not have CKD. What (very simple, no machine learning needed) model can I create that will never tell a person with CKD that they do not have CKD?

> Hint: Don't think about `statsmodels` or `scikit-learn` here.

Answer: We can use the case study and utilise FDP and BP.

### 7. In problem 6, what common classification metric did we optimize for? Did we minimize false positives or negatives?

Answer: We optimize for Specificity. We minimize false negative

### 8. Thinking ethically, what is at least one disadvantage to the model you described in problem 6?

Answer: The model is unable to minimise false positive

### 9. Suppose that I want to construct a model where a person who does not have CKD will ever be told that they do have CKD. What (very simple, no machine learning needed) model can I create that will accomplish this?

Answer: We can use K-Nearest Neighbors

### 10. In problem 9, what common classification metric did we optimize for? Did we minimize false positives or negatives?

Answer: We minimize false positive

### 11. Thinking ethically, what is at least one disadvantage to the model you described in problem 9?

Answer: We are unable to predict true negative.

### 12. Construct a logistic regression model in `sklearn` predicting class from the other variables. You may scale, select/drop, and engineer features as you wish - build a good model! Make sure, however, that you include at least one categorical/dummy feature and at least one quantitative feature.

> Hint: Remember to do a train/test split!

In [38]:
percent_null = (df.isnull().sum() / len(df)).sort_values(ascending=False)
percent_null

rbc      0.3800
rbcc     0.3275
wbcc     0.2650
pot      0.2200
sod      0.2175
pcv      0.1775
pc       0.1625
hemo     0.1300
su       0.1225
sg       0.1175
al       0.1150
bgr      0.1100
bu       0.0475
sc       0.0425
bp       0.0300
age      0.0225
ba       0.0100
pcc      0.0100
htn      0.0050
dm       0.0050
cad      0.0050
appet    0.0025
pe       0.0025
ane      0.0025
class    0.0000
dtype: float64

In [39]:
percent_null[percent_null > 0.1]

rbc     0.3800
rbcc    0.3275
wbcc    0.2650
pot     0.2200
sod     0.2175
pcv     0.1775
pc      0.1625
hemo    0.1300
su      0.1225
sg      0.1175
al      0.1150
bgr     0.1100
dtype: float64

In [40]:
mylist = percent_null[percent_null > 0.1].index
mylist

Index(['rbc', 'rbcc', 'wbcc', 'pot', 'sod', 'pcv', 'pc', 'hemo', 'su', 'sg',
       'al', 'bgr'],
      dtype='object')

In [41]:
df.drop(mylist, axis=1, inplace=True)

In [42]:
percent_null = (df.isnull().sum() / len(df)).sort_values(ascending=False)
percent_null

bu       0.0475
sc       0.0425
bp       0.0300
age      0.0225
pcc      0.0100
ba       0.0100
htn      0.0050
dm       0.0050
cad      0.0050
appet    0.0025
pe       0.0025
ane      0.0025
class    0.0000
dtype: float64

In [43]:
percent_null = (df.isnull().sum() / len(df)).sort_values(ascending=False)
percent_null

bu       0.0475
sc       0.0425
bp       0.0300
age      0.0225
pcc      0.0100
ba       0.0100
htn      0.0050
dm       0.0050
cad      0.0050
appet    0.0025
pe       0.0025
ane      0.0025
class    0.0000
dtype: float64

In [44]:
df[percent_null.index]

,bu,sc,bp,age,pcc,ba,htn,dm,cad,appet,pe,ane,class
0,36.0,1.2,80.0,48.0,notpresent,notpresent,yes,yes,no,good,no,no,ckd
1,18.0,0.8,50.0,7.0,notpresent,notpresent,no,no,no,good,no,no,ckd
2,53.0,1.8,80.0,62.0,notpresent,notpresent,no,yes,no,poor,no,yes,ckd
3,56.0,3.8,70.0,48.0,present,notpresent,yes,no,no,poor,yes,yes,ckd
4,26.0,1.4,80.0,51.0,notpresent,notpresent,no,no,no,good,no,no,ckd
...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,49.0,0.5,80.0,55.0,notpresent,notpresent,no,no,no,good,no,no,notckd
396,31.0,1.2,70.0,42.0,notpresent,notpresent,no,no,no,good,no,no,notckd
397,26.0,0.6,80.0,12.0,notpresent,notpresent,no,no,no,good,no,no,notckd
398,50.0,1.0,60.0,17.0,notpresent,notpresent,no,no,no,good,no,no,notckd


In [45]:
df['bu'].fillna(df['bu'].mean(), inplace=True)
df['sc'].fillna(df['sc'].mean(), inplace=True)
df['bp'].fillna(df['bp'].mean(), inplace=True)
df['age'].fillna(df['age'].mean(), inplace=True)
df['pcc'].fillna('notpresent', inplace=True)
df['ba'].fillna('notpresent', inplace=True)
percent_null = (df.isnull().sum() / len(df)).sort_values(ascending=False)
percent_null

htn      0.0050
dm       0.0050
cad      0.0050
appet    0.0025
pe       0.0025
ane      0.0025
age      0.0000
bp       0.0000
pcc      0.0000
ba       0.0000
bu       0.0000
sc       0.0000
class    0.0000
dtype: float64

In [46]:
mylist = percent_null[percent_null > 0.001].index
mylist
df.drop(mylist, axis=1, inplace=True)

In [47]:
df.head()

,age,bp,pcc,ba,bu,sc,class
0,48.0,80.0,notpresent,notpresent,36.0,1.2,ckd
1,7.0,50.0,notpresent,notpresent,18.0,0.8,ckd
2,62.0,80.0,notpresent,notpresent,53.0,1.8,ckd
3,48.0,70.0,present,notpresent,56.0,3.8,ckd
4,51.0,80.0,notpresent,notpresent,26.0,1.4,ckd


In [48]:
df['class'] = df['class'].apply(lambda x: 1 if x == 'ckd' else 0)
df.head()

,age,bp,pcc,ba,bu,sc,class
0,48.0,80.0,notpresent,notpresent,36.0,1.2,1
1,7.0,50.0,notpresent,notpresent,18.0,0.8,1
2,62.0,80.0,notpresent,notpresent,53.0,1.8,1
3,48.0,70.0,present,notpresent,56.0,3.8,1
4,51.0,80.0,notpresent,notpresent,26.0,1.4,1


In [49]:
df.select_dtypes(include='object').columns
get_dummies_columns = pd.get_dummies(df[df.select_dtypes(include='object').columns])
get_dummies_columns

,pcc_notpresent,pcc_present,ba_notpresent,ba_present
0,1,0,1,0
1,1,0,1,0
2,1,0,1,0
3,0,1,1,0
4,1,0,1,0
...,...,...,...,...
395,1,0,1,0
396,1,0,1,0
397,1,0,1,0
398,1,0,1,0


In [50]:
df.drop(['pcc', 'ba'], axis=1, inplace=True)
df

,age,bp,bu,sc,class
0,48.0,80.0,36.0,1.2,1
1,7.0,50.0,18.0,0.8,1
2,62.0,80.0,53.0,1.8,1
3,48.0,70.0,56.0,3.8,1
4,51.0,80.0,26.0,1.4,1
...,...,...,...,...,...
395,55.0,80.0,49.0,0.5,0
396,42.0,70.0,31.0,1.2,0
397,12.0,80.0,26.0,0.6,0
398,17.0,60.0,50.0,1.0,0


In [51]:
df = pd.concat([get_dummies_columns, df], axis=1)
df

,pcc_notpresent,pcc_present,ba_notpresent,ba_present,age,bp,bu,sc,class
0,1,0,1,0,48.0,80.0,36.0,1.2,1
1,1,0,1,0,7.0,50.0,18.0,0.8,1
2,1,0,1,0,62.0,80.0,53.0,1.8,1
3,0,1,1,0,48.0,70.0,56.0,3.8,1
4,1,0,1,0,51.0,80.0,26.0,1.4,1
...,...,...,...,...,...,...,...,...,...
395,1,0,1,0,55.0,80.0,49.0,0.5,0
396,1,0,1,0,42.0,70.0,31.0,1.2,0
397,1,0,1,0,12.0,80.0,26.0,0.6,0
398,1,0,1,0,17.0,60.0,50.0,1.0,0


In [52]:
X = df.drop('class', axis=1)
y = df['class']

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42, stratify=y)

In [54]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
logreg.score(X_train,y_train)

C:\Users\lydra\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8633333333333333

In [55]:
logreg.score(X_test,y_test)

0.89

In [56]:
(abs(logreg.score(X_train,y_train) - logreg.score(X_test,y_test)) / logreg.score(X_test,y_test)) * 100

2.9962546816479465

---

## Step 5: Evaluate the model.

### 13. Based on your logistic regression model constructed in problem 12, interpret the coefficient of one of your quantitative features.

In [57]:
logreg.coef_

array([[-1.01633737,  0.96896135, -0.72849909,  0.68112307,  0.00247674,
         0.04229649, -0.00756141,  1.8630595 ]])

In [58]:
logreg.coef_[0][4]

0.002476739087057752

In [59]:
np.exp(logreg.coef_[0][4])

1.0024798087390296

logreg.coef_[0][4] is the coefficident for age, therefore a one year increase in age will result in the person increasing his/her chance of obtaining CKD by 1.

### 14. Based on your logistic regression model constructed in problem 12, interpret the coefficient of one of your categorical/dummy features.

In [60]:
logreg.coef_[0][0]

-1.0163373709210408

In [61]:
np.exp(logreg.coef_[0][0])

0.3619180873061128

The presence of pus cells clumps will result in the individual's chance of getting CKD by 0.36.

### 15. Despite being a relatively simple model, logistic regression is very widely used in the real world. Why do you think that's the case? Name at least two advantages to using logistic regression as a modeling technique.

Answer: The first advantage is it is easy to utilise. The second advantage is it can be implemented on multiple features.

### 16. Does it make sense to generate a confusion matrix on our training data or our test data? Why? Generate it on the proper data.

> Hint: Once you've generated your predicted $y$ values and you have your observed $y$ values, then it will be easy to [generate a confusion matrix using sklearn](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html).

It makes sense to generate a confusion matrix on our test data instead of our training data. This is because our model has already seen the training data but not the test data.

In [62]:
y_pred = logreg.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, columns = ['Predict no CKD','Predict CKD'], index=['Actual no CKD', 'Actual CKD'])
cm_df

,Predict no CKD,Predict CKD
Actual no CKD,38,0
Actual CKD,11,51


### 17. In this hospital case, we want to predict CKD. Do we want to optimize for sensitivity, specificity, or something else? Why? (If you don't think there's one clear answer, that's okay! There rarely is. Be sure to defend your conclusion!)

Answer: We want to optimize for sensitivity because it informs us of the ratio of true postivies relative to predicted number of positives.

### 18 (BONUS). Write a function that will create an ROC curve for you, then plot the ROC curve.

Here's a strategy you might consider:
1. In order to even begin, you'll need some fit model. Use your logistic regression model from problem 12.
2. We want to look at all values of your "threshold" - that is, anything where .predict() gives you above your threshold falls in the "positive class," and anything that is below your threshold falls in the "negative class." Start the threshold at 0.
3. At this value of your threshold, calculate the sensitivity and specificity. Store these values.
4. Increment your threshold by some "step." Maybe set your step to be 0.01, or even smaller.
5. At this value of your threshold, calculate the sensitivity and specificity. Store these values.
6. Repeat steps 3 and 4 until you get to the threshold of 1.
7. Plot the values of sensitivity and 1 - specificity.

### 19. Suppose you're speaking with the biostatistics lead at Mayo Clinic, who asks you "Why are unbalanced classes generally a problem? Are they a problem in this particular CKD analysis?" How would you respond?

Answer:

### 20. Suppose you're speaking with a doctor at Mayo Clinic who, despite being very smart, doesn't know much about data science or statistics. How would you explain why unbalanced classes are generally a problem to this doctor?

Answer:

### 21. Let's create very unbalanced classes just for the sake of this example! Generate very unbalanced classes by [bootstrapping](http://stattrek.com/statistics/dictionary.aspx?definition=sampling_with_replacement) (a.k.a. random sampling with replacement) the majority class.

1. The majority class are those individuals with CKD.
2. Generate a random sample of size 200,000 of individuals who have CKD **with replacement**. (Consider setting a random seed for this part!)
3. Create a new dataframe with the original data plus this random sample of data.
4. Now we should have a dataset with around 200,000 observations, of which only about 0.00075% are non-CKD individuals.

### 22. Build a logistic regression model on the unbalanced class data and evaluate its performance using whatever method(s) you see fit. How would you describe the impact of unbalanced classes on logistic regression as a classifier?
> Be sure to look at how well it performs on non-CKD data.

---

## Step 6: Answer the problem.

At this step, you would generally answer the problem! In this situation, you would likely present your model to doctors or administrators at the hospital and show how your model results in reduced false positives/false negatives. Next steps would be to find a way to roll this model and its conclusions out across the hospital so that the outcomes of patients with CKD (and without CKD!) can be improved!